In [2]:
import re
import pandas as pd
from bs4 import BeautifulSoup

# Prices Data

In [4]:
with open('MAX1270.html', 'r') as f:
  contents = f.read()
  soup = BeautifulSoup(contents)

# Preprocessing 

## Dates

In [6]:
def clean_date(date: str) -> str:
  a = re.sub('\n\s{10}', '', date)
  b = re.sub('\n\s{8}', '', a)
  return b

In [5]:
date_regex = re.compile(r'\w{3} \d{2}, \d{4}')
dates = soup.find_all(text = date_regex)
len(dates)

3570

In [81]:
mobile_only_divs[0].find_all(text = date_regex)

[]

In [133]:
content = soup.select('#priceHistoryTable > panel > div > section > section > sortable-table-list > div > sortable-table-list-item > div > span > div.container.container-for-one-items.tabletDesktopOnly > p > span.attribute-value')

In [134]:
# content[0:5]

In [135]:
dates = [x.find_all(text = date_regex) for x in content]
dates = list(filter(lambda x: len(x) > 0, dates))
dates = [item for sublist in dates for item in sublist]

In [136]:
clean_dates = pd.Series(dates).apply(clean_date)
clean_dates.head()

0    May 10, 2011
1    May 11, 2011
2    May 12, 2011
3    May 13, 2011
4    May 16, 2011
dtype: object

In [137]:
clean_dates.tail(1)

1784    Apr 30, 2021
dtype: object

In [149]:
len(clean_dates)

3570

## Prices

In [139]:
def clean_price(price: str) -> str:
  a = re.sub('\n\s{10}', '', price)
  b = re.sub('\n\s{8}', '', a)
  return b

In [140]:
price_regex = re.compile(r'\$\d.*\.\d{4}')
prices = soup.find_all(text = price_regex)
prices = pd.Series(prices).apply(clean_price)
len(prices)

5019

In [141]:
prices = [x.find_all(text = price_regex) for x in content]
prices = list(filter(lambda x: len(x) > 0, prices))
prices = [item for sublist in prices for item in sublist]

In [142]:
clean_prices = pd.Series(prices).apply(clean_price)
clean_prices.head(10)

0    $11.4020
1    $11.3745
2    $11.3705
3    $11.3779
4    $11.3938
5    $11.3967
6    $11.3998
7    $11.4087
8    $11.4316
9    $11.4231
dtype: object

In [150]:
len(clean_prices)*2

5018

In [144]:
market_data = pd.concat([
  clean_dates,
  clean_prices
], axis = 1, ignore_index = True).rename(columns = {
  0: 'date',
  1: 'price'
})

market_data.query('date.isnull()').head(1)

,date,price
1785,NaN,$13.0745


In [132]:
market_data.sample(1)

,date,price
1610,"May 13, 2020",$12.7683


In [146]:
market_data.iloc[1780]

date     Apr 26, 2021
price        $12.9561
Name: 1780, dtype: object

In [148]:
market_data.query('price == "$13.5752"')

,date,price
2499,NaN,$13.5752
